In [1]:
%load_ext autoreload
%autoreload 2

env = 'prod'
pk_project = 6857901
execute = True
metadata_str = 'bhp-actors-births'
import_manner = 'one-shot'

import os
import pandas as pd
import numpy as np
from datetime import datetime
import duckdb
import plotly.express as px

import geovpylib.analysis as a
import geovpylib.database as db
import geovpylib.graphs as graphs
import geovpylib.pks as pks
import geovpylib.recordlinkage as rl
import geovpylib.sparql as sparql
import geovpylib.utils as u

eta = u.Eta()

# Date completion

Because of this [comment](https://github.com/geovistory/project-symogih/issues/1#issuecomment-1485216923), some births/deaths does not have a date. The goal of this notebook is to add them.

## Births

### Fetch information on BHP

We take the logic we had at the first place, but keep certainty_begin == 3 or 2

In [2]:
db.connect_external(os.environ.get('YELLOW_BHP'))

actors = db.query('select * from bhp.actor')

actors = actors[~actors['concat_standard_name'].str.contains('\[à identifier\]')]
actors = actors[~actors['concat_standard_name'].str.contains('\[ne pas importer\]')]
actors = actors[~actors['concat_standard_name'].str.contains('DOUBLON')]
actors = actors[~actors['concat_standard_name'].str.contains('Doublon')]
actors = actors[~actors['concat_standard_name'].str.contains('réutiliser')]
actors = actors[~actors['concat_standard_name'].str.contains('REUTILISER')]
actors = actors[actors['fk_abob_type_actor'] == 104]
actors.replace({'\r\n':''}, regex=True, inplace=True)
actors.replace({'<p>':''}, regex=True, inplace=True)
actors.replace({'</p>':''}, regex=True, inplace=True)
actors['gender_iso'].replace(pd.NA, 0, inplace=True)
actors['certainty_begin'].replace(pd.NA, '0', inplace=True)
actors['certainty_end'].replace(' ', pd.NA, inplace=True)
actors['certainty_end'].replace(pd.NA, '0', inplace=True)
for i, row in actors.iterrows():
    if pd.isna(row['begin_year']): actors.at[i, 'certainty_begin'] = '0'
    if pd.isna(row['end_year']): actors.at[i, 'certainty_end'] = '0'


actors = actors[(actors['certainty_begin'] == '3') | (actors['certainty_begin'] == '2')]
actors['begin_year'] = actors['begin_year'].astype(int)
actors['date'] = [(y, 1, 1) for y in actors.begin_year]
actors = actors[['pk_actor', 'date', 'certainty_begin']]
actors.columns = ['pk_bhp', 'date', 'certainty']

a.infos(actors)

# 5s

[DB] Connecting to PGSQL Database ... Connected!
Shape:  (16350, 3) - extract:


,pk_bhp,date,certainty
0,44895,"(1870, 1, 1)",3
5,4040,"(1621, 1, 1)",3
14,12072,"(1830, 1, 1)",3
22,30861,"(1631, 1, 1)",3
31,25185,"(1849, 1, 1)",3


### Fetch information on Geovistory

In [3]:
db.connect_geovistory(env, pk_project, execute, skip_protection=True)

persons = db.query(f"""
    select 
        r.pk_entity as pk_person
        ,a3.string as uri
        ,s4.fk_subject_info as pk_birth
    from information.resource r 
    inner join projects.info_proj_rel ipr on ipr.fk_entity = r.pk_entity and ipr.fk_project = {pk_project} and ipr.is_in_project = true
    -- URI
    inner join information.statement s1 on s1.fk_subject_info = r.pk_entity and s1.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = s1.fk_object_info and s2.fk_property = {pks.properties.appe_hasValue_string}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.appellation a3 on a3.pk_entity = s2.fk_object_info
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = a3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    -- Birth
    inner join information.statement s4 on s4.fk_object_info = r.pk_entity and s4.fk_property = {pks.properties.birth_broughtIntoLife_person}
    inner join projects.info_proj_rel ipr4 on ipr4.fk_entity = s4.pk_entity and ipr4.fk_project = {pk_project} and ipr4.is_in_project = true
    inner join projects.info_proj_rel ipr4b on ipr4b.fk_entity = s4.fk_subject_info and ipr4b.fk_project = {pk_project} and ipr4b.is_in_project = true
    where r.fk_class = {pks.classes.person}
""")

# URI
persons = persons[persons.uri.str.contains('symogih.org')]
persons['pk_bhp'] = persons.uri.str.replace('http://symogih.org/resource/Actr', '', regex=False).astype(int)
persons.drop(columns=['uri'], inplace=True)

# Reset
persons.sort_values('pk_bhp', inplace=True)
persons.drop_duplicates(inplace=True)
persons.reset_index(inplace=True, drop=True)

a.infos(persons)
db.disconnect()

# 10s

[DB] Connecting to PRODUCTION Database ... Connected!
Shape:  (59525, 3) - extract:


,pk_person,pk_birth,pk_bhp
0,6532778,7489054,1
1,6499432,7489055,2
2,783602,806773,3
3,6509333,7489056,4
4,6511070,7489057,5


[DB] Database correctly disconnected.


In [4]:
db.connect_geovistory(env, pk_project, execute, skip_protection=True)

births = db.query(f"""
    select
        r.pk_entity as pk_birth
        ,tp1.julian_day, tp1.calendar        
        ,a3.string as current_comment
    from information.resource r
    inner join projects.info_proj_rel ipr on ipr.fk_entity = r.pk_entity and ipr.fk_project = {pk_project} and ipr.is_in_project = true
    -- birth date
    inner join information.statement s1 on s1.fk_subject_info = r.pk_entity and (s1.fk_property = {pks.properties.timeSpan_atSomeTimeWithin_timePrimitive} or s1.fk_property = {pks.properties.timespan_beginOfTheBegin_timePrim} or s1.fk_property = {pks.properties.timespan_endOfTheEnd_timePrim})
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.time_primitive tp1 on tp1.pk_entity = s1.fk_object_info
    -- comment
    left join information.statement s2 on s2.fk_subject_info = s1.fk_subject_info and s2.fk_property = {pks.properties.entity_hasComment_text}
    left join information.statement s3 on s3.fk_subject_info = s2.fk_object_info and s3.fk_property = {pks.properties.text_hasValueVersion_string}
    left join information.appellation a3 on a3.pk_entity = s3.fk_object_info
    where r.fk_class = {pks.classes.birth}
""")

births['date_existing'] = [u.from_julian_day(row['julian_day'], row['calendar']) for _,row in births.iterrows()]
births.drop(columns=['julian_day', 'calendar'], inplace=True)


a.infos(births)
db.disconnect()

# 3s

[DB] Connecting to PRODUCTION Database ... Connected!
Shape:  (37385, 3) - extract:


,pk_birth,current_comment,date_existing
0,7489073,None,"(1647, 1, 1)"
1,7489054,None,"(1599, 1, 1)"
2,7489055,None,"(1600, 1, 1)"
3,7489056,None,"(1600, 1, 1)"
4,7489057,None,"(1681, 1, 1)"


[DB] Database correctly disconnected.


In [5]:
persons = persons.merge(births, on='pk_birth', how='left')

a.infos(persons)

Shape:  (59866, 5) - extract:


,pk_person,pk_birth,pk_bhp,current_comment,date_existing
0,6532778,7489054,1,None,"(1599, 1, 1)"
1,6499432,7489055,2,None,"(1600, 1, 1)"
2,783602,806773,3,None,"(1581, 1, 1)"
3,6509333,7489056,4,None,"(1600, 1, 1)"
4,6511070,7489057,5,None,"(1681, 1, 1)"


### Data merge

In [6]:
table = actors.merge(persons, on='pk_bhp')
table = table[pd.isna(table['date_existing'])]
table['comment'] = ['Date postulée' if row['certainty'] == '2' else 'Date reconstituée' for _,row in table.iterrows()]

a.infos(table, random=True)

Shape:  (11438, 7) - extract:


,pk_bhp,date,certainty,pk_person,pk_birth,current_comment,date_existing
951,42443,"(1887, 1, 1)",3,6494674,7527651,NaN,NaN
741,31455,"(1880, 1, 1)",3,6494342,7518767,NaN,NaN
13020,60962,"(1825, 1, 1)",3,6535972,7544799,NaN,NaN
14822,46241,"(1625, 1, 1)",3,6539719,7531369,NaN,NaN
13416,61035,"(1825, 1, 1)",3,6537033,7544871,NaN,NaN


In [7]:
table2 = actors.merge(persons, on='pk_bhp')
table2 = table2[table2['certainty'] == '2']
table2 = table2[pd.isna(table2['current_comment'])]
table2['comment'] = ['Date postulée' if row['certainty'] == '2' else '' for _,row in table2.iterrows()]

a.infos(table2, random=True)

Shape:  (899, 8) - extract:


,pk_bhp,date,certainty,pk_person,pk_birth,current_comment,date_existing,comment
9652,42206,"(1876, 1, 1)",2,6522925,7527419,None,"(1876, 1, 1)",Date postulée
10468,42068,"(1879, 1, 1)",2,6528000,7527281,None,"(1879, 1, 1)",Date postulée
9698,42261,"(1878, 1, 1)",2,6522980,7527474,None,"(1878, 1, 1)",Date postulée
9693,42256,"(1873, 1, 1)",2,6522974,7527469,None,"(1873, 1, 1)",Date postulée
9362,40108,"(1901, 1, 1)",2,6521290,7525606,None,"(1901, 1, 1)",Date postulée


### Create information

In [9]:
db.connect_geovistory(env, pk_project, execute)

[DB] Connecting to PRODUCTION Database ... Connected!


**Birth date**

In [11]:
table['pk_tp'] = db.time_primitives.create(table['date'], '1 year')
db.statements.create(table['pk_birth'], pks.properties.timeSpan_atSomeTimeWithin_timePrimitive, table['pk_tp'])

Creating 11438 time primitives ... Done in [00h00'01]
Creating info_proj_rel of 11438 entities with project <6857901> ... Done in [00h00'05]
Creating 11438 statements ... Done in [00h00'10]
Creating info_proj_rel of 11438 entities with project <6857901> ... Done in [00h00'04]


In [ ]:
# db.connect_geovistory(env, pk_project, execute, skip_protection=True)
# values = '(' + ','.join([str(e) for e in table.pk_birth]) + ')'

# r = db.query(f"""
#     select 
#         s.fk_subject_info, s.fk_property, s.fk_object_info
#     from information.statement s
#     inner join projects.info_proj_rel ipr on ipr.fk_entity = s.pk_entity and ipr.fk_project = {pk_project} and ipr.is_in_project = true
#     where s.fk_subject_info in {values} and 
#        (s.fk_property = {pks.properties.timeSpan_atSomeTimeWithin_timePrimitive}
#     or  s.fk_property = {pks.properties.timespan_beginOfTheBegin_timePrim}
#     or  s.fk_property = {pks.properties.timespan_endOfTheEnd_timePrim} )
    
# """)

# db.disconnect()

# r

**Comments**: "Reconstituée"

In [17]:
pk_intance_comment_type = 7953586

# table['pk_comment'] = db.resources.create(pks.classes.text, len(table))
table['pk_appe'] = db.appellations.create(table['comment'])

db.statements.create(table['pk_birth'], pks.properties.entity_hasComment_text, table['pk_comment'])
db.statements.create(table['pk_comment'], pks.properties.text_hasTextType_textType, pk_intance_comment_type)
db.statements.create(table['pk_comment'], pks.properties.text_hasValueVersion_string, table['pk_appe'])

Creating 11438 appellations ... Done in [00h00'04]
Creating 11438 statements ... Done in [00h00'09]
Creating info_proj_rel of 11438 entities with project <6857901> ... Done in [00h00'05]
Creating 11438 statements ... Done in [00h00'08]
Creating info_proj_rel of 11438 entities with project <6857901> ... Done in [00h00'03]
Creating 11438 statements ... Done in [00h00'09]
Creating info_proj_rel of 11438 entities with project <6857901> ... Done in [00h00'04]


**Comments**: "Postulée"

In [18]:
pk_intance_comment_type = 7953586

table2['pk_comment'] = db.resources.create(pks.classes.text, len(table2))
table2['pk_appe'] = db.appellations.create(table2['comment'])

db.statements.create(table2['pk_birth'], pks.properties.entity_hasComment_text, table2['pk_comment'])
db.statements.create(table2['pk_comment'], pks.properties.text_hasTextType_textType, pk_intance_comment_type)
db.statements.create(table2['pk_comment'], pks.properties.text_hasValueVersion_string, table2['pk_appe'])

Creating 899 resources of class [785] ... Done in [00h00'00]
Creating info_proj_rel of 899 entities with project <6857901> ... Done in [00h00'01]
Creating 899 appellations ... Done in [00h00'00]
Creating 899 statements ... Done in [00h00'01]
Creating info_proj_rel of 899 entities with project <6857901> ... Done in [00h00'00]
Creating 899 statements ... Done in [00h00'01]
Creating info_proj_rel of 899 entities with project <6857901> ... Done in [00h00'00]
Creating 899 statements ... Done in [00h00'01]
Creating info_proj_rel of 899 entities with project <6857901> ... Done in [00h00'01]


In [19]:
db.disconnect()

[DB] Database correctly disconnected.


## Deaths

### Fetch information on BHP

We take the logic we had at the first place, but keep certainty_end == 3 or 2

In [20]:
db.connect_external(os.environ.get('YELLOW_BHP'))

actors = db.query('select * from bhp.actor')

actors = actors[~actors['concat_standard_name'].str.contains('\[à identifier\]')]
actors = actors[~actors['concat_standard_name'].str.contains('\[ne pas importer\]')]
actors = actors[~actors['concat_standard_name'].str.contains('DOUBLON')]
actors = actors[~actors['concat_standard_name'].str.contains('Doublon')]
actors = actors[~actors['concat_standard_name'].str.contains('réutiliser')]
actors = actors[~actors['concat_standard_name'].str.contains('REUTILISER')]
actors = actors[actors['fk_abob_type_actor'] == 104]
actors.replace({'\r\n':''}, regex=True, inplace=True)
actors.replace({'<p>':''}, regex=True, inplace=True)
actors.replace({'</p>':''}, regex=True, inplace=True)
actors['gender_iso'].replace(pd.NA, 0, inplace=True)
actors['certainty_begin'].replace(pd.NA, '0', inplace=True)
actors['certainty_end'].replace(' ', pd.NA, inplace=True)
actors['certainty_end'].replace(pd.NA, '0', inplace=True)
for i, row in actors.iterrows():
    if pd.isna(row['begin_year']): actors.at[i, 'certainty_begin'] = '0'
    if pd.isna(row['end_year']): actors.at[i, 'certainty_end'] = '0'


actors = actors[(actors['certainty_end'] == '3') | (actors['certainty_end'] == '2')]
actors['end_year'] = actors['end_year'].astype(int)
actors['date'] = [(y, 1, 1) for y in actors.end_year]
actors = actors[['pk_actor', 'date', 'certainty_end']]
actors.columns = ['pk_bhp', 'date', 'certainty']

a.infos(actors)

# 5s

[DB] Connecting to PGSQL Database ... Connected!
Shape:  (3326, 3) - extract:


,pk_bhp,date,certainty
0,44895,"(1930, 1, 1)",3
1,47015,"(1545, 1, 1)",3
542,47606,"(1582, 1, 1)",3
624,30877,"(1640, 1, 1)",3
3462,16798,"(1684, 1, 1)",3


### Fetch information on Geovistory

In [21]:
db.connect_geovistory(env, pk_project, execute, skip_protection=True)

persons = db.query(f"""
    select 
        r.pk_entity as pk_person
        ,a3.string as uri
        ,s4.fk_subject_info as pk_death
    from information.resource r 
    inner join projects.info_proj_rel ipr on ipr.fk_entity = r.pk_entity and ipr.fk_project = {pk_project} and ipr.is_in_project = true
    -- URI
    inner join information.statement s1 on s1.fk_subject_info = r.pk_entity and s1.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = s1.fk_object_info and s2.fk_property = {pks.properties.appe_hasValue_string}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.appellation a3 on a3.pk_entity = s2.fk_object_info
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = a3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    -- Death
    inner join information.statement s4 on s4.fk_object_info = r.pk_entity and s4.fk_property = {pks.properties.death_wasDeathOf_person}
    inner join projects.info_proj_rel ipr4 on ipr4.fk_entity = s4.pk_entity and ipr4.fk_project = {pk_project} and ipr4.is_in_project = true
    inner join projects.info_proj_rel ipr4b on ipr4b.fk_entity = s4.fk_subject_info and ipr4b.fk_project = {pk_project} and ipr4b.is_in_project = true
    where r.fk_class = {pks.classes.person}
""")

# URI
persons = persons[persons.uri.str.contains('symogih.org')]
persons['pk_bhp'] = persons.uri.str.replace('http://symogih.org/resource/Actr', '', regex=False).astype(int)
persons.drop(columns=['uri'], inplace=True)

# Reset
persons.sort_values('pk_bhp', inplace=True)
persons.drop_duplicates(inplace=True)
persons.reset_index(inplace=True, drop=True)

a.infos(persons)
db.disconnect()

# 5s

[DB] Connecting to PRODUCTION Database ... Connected!
Shape:  (59525, 3) - extract:


,pk_person,pk_death,pk_bhp
0,6532778,7726515,1
1,6499432,7726516,2
2,783602,7726517,3
3,6509333,7726518,4
4,6511070,7726519,5


[DB] Database correctly disconnected.


In [23]:
db.connect_geovistory(env, pk_project, execute, skip_protection=True)

deaths = db.query(f"""
    select
        r.pk_entity as pk_death
        ,tp1.julian_day, tp1.calendar        
        ,a3.string as current_comment
    from information.resource r
    inner join projects.info_proj_rel ipr on ipr.fk_entity = r.pk_entity and ipr.fk_project = {pk_project} and ipr.is_in_project = true
    -- death date
    inner join information.statement s1 on s1.fk_subject_info = r.pk_entity and (s1.fk_property = {pks.properties.timeSpan_atSomeTimeWithin_timePrimitive} or s1.fk_property = {pks.properties.timespan_beginOfTheBegin_timePrim} or s1.fk_property = {pks.properties.timespan_endOfTheEnd_timePrim})
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.time_primitive tp1 on tp1.pk_entity = s1.fk_object_info
    -- comment
    left join information.statement s2 on s2.fk_subject_info = s1.fk_subject_info and s2.fk_property = {pks.properties.entity_hasComment_text}
    left join information.statement s3 on s3.fk_subject_info = s2.fk_object_info and s3.fk_property = {pks.properties.text_hasValueVersion_string}
    left join information.appellation a3 on a3.pk_entity = s3.fk_object_info
    where r.fk_class = {pks.classes.death}
""")

deaths['date_existing'] = [u.from_julian_day(row['julian_day'], row['calendar']) for _,row in deaths.iterrows()]
deaths.drop(columns=['julian_day', 'calendar'], inplace=True)


a.infos(deaths)
db.disconnect()

[DB] Connecting to PRODUCTION Database ... Connected!
Shape:  (26376, 3) - extract:


,pk_death,current_comment,date_existing
0,7727432,None,"(1659, 1, 1)"
1,7727415,None,"(1596, 1, 1)"
2,7727416,None,"(1672, 1, 1)"
3,7727417,None,"(1582, 1, 1)"
4,7727418,None,"(1678, 1, 1)"


[DB] Database correctly disconnected.


In [24]:
persons = persons.merge(deaths, on='pk_death', how='left')

a.infos(persons)

Shape:  (59553, 5) - extract:


,pk_person,pk_death,pk_bhp,current_comment,date_existing
0,6532778,7726515,1,None,"(1653, 1, 1)"
1,6499432,7726516,2,None,"(1672, 1, 1)"
2,783602,7726517,3,None,"(1643, 1, 1)"
3,6509333,7726518,4,None,"(1635, 1, 1)"
4,6511070,7726519,5,None,"(1746, 1, 1)"


### Data merge

In [27]:
table = actors.merge(persons, on='pk_bhp')
table = table[pd.isna(table['date_existing'])]
table['comment'] = ['Date postulée' if row['certainty'] == '2' else 'Date reconstituée' for _,row in table.iterrows()]

a.infos(table, random=True)

Shape:  (3041, 8) - extract:


,pk_bhp,date,certainty,pk_person,pk_death,current_comment,date_existing,comment
2501,61384,"(1901, 1, 1)",3,6538089,7783823,NaN,NaN,Date reconstituée
2313,61192,"(1901, 1, 1)",3,6537661,7783631,NaN,NaN,Date reconstituée
501,26729,"(1862, 1, 1)",3,6513056,7752530,NaN,NaN,Date reconstituée
2487,46295,"(1695, 1, 1)",3,6538048,7769205,NaN,NaN,Date reconstituée
692,27273,"(1896, 1, 1)",3,6513469,7752960,NaN,NaN,Date reconstituée


In [28]:
table2 = actors.merge(persons, on='pk_bhp')
table2 = table2[table2['certainty'] == '2']
table2 = table2[pd.isna(table2['current_comment'])]
table2['comment'] = ['Date postulée' if row['certainty'] == '2' else '' for _,row in table2.iterrows()]

a.infos(table2, random=True)

Shape:  (150, 8) - extract:


,pk_bhp,date,certainty,pk_person,pk_death,current_comment,date_existing,comment
1903,57909,"(1999, 1, 1)",2,6533919,7780339,None,"(1999, 1, 1)",Date postulée
3176,46209,"(1695, 1, 1)",2,6542029,7769119,None,"(1695, 1, 1)",Date postulée
2511,59543,"(1527, 1, 1)",2,6538150,7781975,None,"(1527, 1, 1)",Date postulée
1308,38742,"(1631, 1, 1)",2,6519981,7761969,None,"(1631, 1, 1)",Date postulée
1897,57894,"(1955, 1, 1)",2,6533903,7780324,None,"(1955, 1, 1)",Date postulée


### Create information

In [29]:
db.connect_geovistory(env, pk_project, execute)

[DB] Connecting to PRODUCTION Database ... Connected!


**Birth date**

In [30]:
table['pk_tp'] = db.time_primitives.create(table['date'], '1 year')
db.statements.create(table['pk_death'], pks.properties.timeSpan_atSomeTimeWithin_timePrimitive, table['pk_tp'])

Creating 3041 time primitives ... Done in [00h00'00]
Creating 3041 statements ... Done in [00h00'03]
Creating info_proj_rel of 3041 entities with project <6857901> ... Done in [00h00'01]


In [ ]:
# db.connect_geovistory(env, pk_project, execute, skip_protection=True)
# values = '(' + ','.join([str(e) for e in table.pk_birth]) + ')'

# r = db.query(f"""
#     select 
#         s.fk_subject_info, s.fk_property, s.fk_object_info
#     from information.statement s
#     inner join projects.info_proj_rel ipr on ipr.fk_entity = s.pk_entity and ipr.fk_project = {pk_project} and ipr.is_in_project = true
#     where s.fk_subject_info in {values} and 
#        (s.fk_property = {pks.properties.timeSpan_atSomeTimeWithin_timePrimitive}
#     or  s.fk_property = {pks.properties.timespan_beginOfTheBegin_timePrim}
#     or  s.fk_property = {pks.properties.timespan_endOfTheEnd_timePrim} )
    
# """)

# db.disconnect()

# r

**Comments**: "Reconstituée"

In [31]:
pk_intance_comment_type = 7953586

table['pk_comment'] = db.resources.create(pks.classes.text, len(table))
table['pk_appe'] = db.appellations.create(table['comment'])

db.statements.create(table['pk_death'], pks.properties.entity_hasComment_text, table['pk_comment'])
db.statements.create(table['pk_comment'], pks.properties.text_hasTextType_textType, pk_intance_comment_type)
db.statements.create(table['pk_comment'], pks.properties.text_hasValueVersion_string, table['pk_appe'])

Creating 3041 resources of class [785] ... Done in [00h00'00]
Creating info_proj_rel of 3041 entities with project <6857901> ... Done in [00h00'01]
Creating 3041 appellations ... Done in [00h00'02]
Creating 3041 statements ... Done in [00h00'02]
Creating info_proj_rel of 3041 entities with project <6857901> ... Done in [00h00'01]
Creating 3041 statements ... Done in [00h00'03]
Creating info_proj_rel of 3041 entities with project <6857901> ... Done in [00h00'01]
Creating 3041 statements ... Done in [00h00'02]
Creating info_proj_rel of 3041 entities with project <6857901> ... Done in [00h00'02]


**Comments**: "Postulée"

In [32]:
pk_intance_comment_type = 7953586

table2['pk_comment'] = db.resources.create(pks.classes.text, len(table2))
table2['pk_appe'] = db.appellations.create(table2['comment'])

db.statements.create(table2['pk_death'], pks.properties.entity_hasComment_text, table2['pk_comment'])
db.statements.create(table2['pk_comment'], pks.properties.text_hasTextType_textType, pk_intance_comment_type)
db.statements.create(table2['pk_comment'], pks.properties.text_hasValueVersion_string, table2['pk_appe'])

Creating 150 resources of class [785] ... Done in [00h00'00]
Creating info_proj_rel of 150 entities with project <6857901> ... Done in [00h00'00]
Creating 150 appellations ... Done in [00h00'00]
Creating 150 statements ... Done in [00h00'01]
Creating info_proj_rel of 150 entities with project <6857901> ... Done in [00h00'00]
Creating 150 statements ... Done in [00h00'00]
Creating info_proj_rel of 150 entities with project <6857901> ... Done in [00h00'00]
Creating 150 statements ... Done in [00h00'00]
Creating info_proj_rel of 150 entities with project <6857901> ... Done in [00h00'01]


In [33]:
db.disconnect()

[DB] Database correctly disconnected.
